# Chemprop scikit-learn Estimator Example
Demonstrating usage of `ChempropTransformer` and `ChempropRegressor` with common scikit-learn workflows including cross-validation, hyperparameter tuning, and model persistence.

In [ ]:
from chemprop.sklearn_integration.chemprop_lightning_estimator import ChempropMoleculeTransformer, ChempropRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV
import numpy as np
import joblib

# Sample data
X = np.array([
    "CCO", "CCN", "CCC", "COC", "CNC", "CCCl", "CCBr", "CCF", "CCI", "CC=O",
    "CC#N", "CC(C)O", "CC(C)N", "CC(C)C", "COC(C)", "CN(C)C", "C1CCCCC1", "C1=CC=CC=C1",
    "CC(C)(C)O", "CC(C)(C)N", "COCCO", "CCOC(=O)C", "CCN(CC)CC", "CN1CCCC1", "C(CO)N"
])
y = np.array([
    0.50, 0.60, 0.55, 0.58, 0.52, 0.62, 0.65, 0.57, 0.59, 0.61,
    0.56, 0.60, 0.54, 0.53, 0.62, 0.63, 0.45, 0.40,
    0.64, 0.66, 0.59, 0.51, 0.48, 0.46, 0.49
])

## Cross-Validation Example

In [ ]:
pipeline = Pipeline([
    ("featurizer", ChempropMoleculeTransformer()),
    ("regressor", ChempropRegressor(n_epochs=20))
])

scores = cross_val_score(pipeline, X, y, cv=5, scoring='neg_mean_squared_error')
print("Cross-validation MSE scores:", -scores)
print("Average MSE:", -scores.mean())

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\jxl05\miniconda3\envs\chemprop1\Lib\site-packages\lightning\pytorch\trainer\configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
c:\Users\jxl05\miniconda3\envs\chemprop1\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:654: Checkpoint directory c:\Users\jxl05\Downloads\chemprop\chemprop\sklearn_integration\checkpoints exists and is not empty.
Loading `train_dataloader` to estimate number of stepping batches.
c:\Users\jxl05\miniconda3\envs\chemprop1\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in t

Epoch 17: 100%|██████████| 1/1 [00:00<00:00, 21.58it/s, train_loss_step=0.000498, train_loss_epoch=0.000498]

c:\Users\jxl05\miniconda3\envs\chemprop1\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.



Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 83.36it/s] 


c:\Users\jxl05\miniconda3\envs\chemprop1\Lib\site-packages\sklearn\pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\jxl05\miniconda3\envs\chemprop1\Lib\site-packages\lightning\pytorch\trainer\configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
c:\Users\jxl05\miniconda3\envs\chemprop1\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:654: Checkpoint directory c:\Users\jxl05\Downloads\chemprop\chemprop\sklearn_integration\checkpoints exists and is not empty.
Loading `train_dataloader` to estimate number of stepping batches.
c:\Users\jxl05\miniconda3\envs\chemprop1\Lib\site-packages\lightning\pyto

...

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 1/1 [00:00<00:00, 25.00it/s, train_loss_step=0.000408, train_loss_epoch=0.000408]

c:\Users\jxl05\miniconda3\envs\chemprop1\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.



Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 73.98it/s]
Cross-validation MSE scores: [0.20029232 0.26940486 0.19254542 0.13431077 0.09734544]
Average MSE: 0.1787797631378953


c:\Users\jxl05\miniconda3\envs\chemprop1\Lib\site-packages\sklearn\pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


## Hyperparameter Tuning Example with GridSearchCV

In [7]:
param_grid = {
    'regressor__batch_size': [8, 16],
    'regressor__n_epochs': [3, 5],
}

grid = GridSearchCV(pipeline, param_grid, cv=3, scoring='neg_mean_squared_error')
grid.fit(X, y)

print("Best parameters:", grid.best_params_)
print("Best score (MSE):", -grid.best_score_)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\jxl05\miniconda3\envs\chemprop1\Lib\site-packages\lightning\pytorch\trainer\configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
c:\Users\jxl05\miniconda3\envs\chemprop1\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:654: Checkpoint directory c:\Users\jxl05\Downloads\chemprop\chemprop\sklearn_integration\checkpoints exists and is not empty.
Loading `train_dataloader` to estimate number of stepping batches.
c:\Users\jxl05\miniconda3\envs\chemprop1\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in t

Epoch 19: 100%|██████████| 2/2 [00:00<00:00, 30.77it/s, train_loss_step=0.00543, train_loss_epoch=0.00396]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 2/2 [00:00<00:00, 28.58it/s, train_loss_step=0.00543, train_loss_epoch=0.00396]

c:\Users\jxl05\miniconda3\envs\chemprop1\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


...

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\jxl05\miniconda3\envs\chemprop1\Lib\site-packages\lightning\pytorch\trainer\configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
c:\Users\jxl05\miniconda3\envs\chemprop1\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:654: Checkpoint directory c:\Users\jxl05\Downloads\chemprop\chemprop\sklearn_integration\checkpoints exists and is not empty.
Loading `train_dataloader` to estimate number of stepping batches.
c:\Users\jxl05\miniconda3\envs\chemprop1\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.

  | Name            | Type               | Params | Mode 
------------

Epoch 0:  50%|█████     | 1/2 [00:00<00:00, 21.03it/s, train_loss_step=0.385]

c:\Users\jxl05\miniconda3\envs\chemprop1\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
8 fits failed out of a total of 12.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
8 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\jxl05\miniconda3\envs\chemprop1\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\jxl05\miniconda3\envs\chemprop1\Lib\site-packages\sklearn\base.py", line 1363, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jxl05\miniconda3\envs\chemprop1\Lib\site-packages\sklear

Epoch 0:  50%|█████     | 1/2 [00:02<00:02,  0.37it/s, train_loss_step=0.230]


ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 300])

## Save and Reload Trained Pipeline

In [5]:
joblib.dump(grid.best_estimator_, 'best_chemprop_pipeline.pkl')
loaded = joblib.load('best_chemprop_pipeline.pkl')
preds = loaded.predict(["CCO", "CC(=O)O"])
print("Predictions from reloaded model:", preds)

Predictions from reloaded model: [0.16074981 0.13957587]


c:\Users\jxl05\miniconda3\envs\chemprop1\Lib\site-packages\sklearn\pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
